# imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob
import re
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# import sys
# sys.path.append('/root/digital-twin/wireless_digital_twin/')

import tensorflow as tf
print(tf.test.gpu_device_name())


import utils.utilfunc as uf
from utils.models import PlanNet, RouteNet
from utils.datagen_old import PlanDataGen, get_output_format
import utils.datagen as dg

2022-12-23 05:57:16.207293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 05:57:16.302343: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-23 05:57:17.412417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriat

/device:GPU:0


# Config Files and Data Loaders

In [2]:
for a,b,c in zip((1,2,3),(1,2,3),(1,2,3)):
    print(a,b,c)

1 1 1
2 2 2
3 3 3


In [3]:
# abidata = '/root/digital-twin/digi_twin_summer_wireless_dataset-e3cce9c3860f1c1f3de8a0dcb5c744bb16ee4f34/data_split_clean'
# abidata2 = '/root/digital-twin/digi_twin_summer_wireless_dataset-e3cce9c3860f1c1f3de8a0dcb5c744bb16ee4f34/dataset-2'

config  = uf.PathConfigParser('./configs/config_combined.ini').as_dict()
hparams = dict(**config['GNN'], **config['LearningParams'])

datagens, datasets = dg.get_data_gens_sets(config)
config

<class 'networkx.utils.decorators.argmap'> compilation 9:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/root/digital-twin/wireless_digital_twin/utils/datagen.py:238: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(self.graph_topology_undirected)
2022-12-23 05:57:18.915582: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5470 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-12-23 05:57:18.915934: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9634 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


{'Paths': {'data': ['./dataset/NSFNet_routing_1',
   './dataset/NSFNet_routing_2'],
  'graph': ['./dataset/nsfnet.txt', './dataset/nsfnet.txt'],
  'routing': ['paths1', 'paths2'],
  'logs': ['setting-2/joint/logs/']},
 'GNN': {'T': 4,
  'link_state_dim': 16,
  'path_state_dim': 32,
  'node_state_dim': 16,
  'readoutLayerSizes': [16, 32, 16],
  'edgeMLPLayerSizes': [8, 16, 32, 16, 8]},
 'LearningParams': {'epochs': 50,
  'learning_rate': 0.0001,
  'lr_decay_steps': 10000,
  'lr_decay_rate': 0.95,
  'batch_size': 32,
  'dropout_rate': 0.1,
  'l2': 0.01,
  'l2_2': 0.01,
  'learn_embedding': True}}

# Models

In [4]:
datagens['train'].kpiframe.shape

(13876, 30)

In [12]:

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

checkpoint_dir = config['Paths']['logs'][0]
print(checkpoint_dir)
os.makedirs(checkpoint_dir, exist_ok = True)
checkpoint_path = os.path.join(checkpoint_dir, "cp.ckpt")
csvlog_path = os.path.join(checkpoint_dir, "training.log")

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1)
cp_csv0 = tf.keras.callbacks.CSVLogger(
    csvlog_path, separator=',', append=True)

model = PlanNet(hparams, train_on = 'delay')
# model = RouteNet(hparams, train_on = 'delay')
model.build()
model.compile(optimizer="adam", run_eagerly=False)


model(datagens['train'].__getitem__(0)[0])


setting-2/joint/logs/
[None, 32] [None, 64] 16


<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-0.6584356 ],
       [-1.0012652 ],
       [-1.8473941 ],
       [-1.3098133 ],
       [-2.3904784 ],
       [-0.67293197],
       [-0.7125792 ],
       [-0.02819516],
       [-1.2243168 ],
       [-2.2916396 ]], dtype=float32)>

In [10]:
config['Paths']['logs']

['setting-2/joint/logs/']

In [8]:
iters_per_epoch = 100
num_loops = 1
model.fit(
    x                = datasets['train'], 
    steps_per_epoch  = datagens['train'].__len__()//iters_per_epoch, 
    epochs           = iters_per_epoch * num_loops, 
    validation_data  = datasets['validate'], 
    validation_steps = datagens['validate'].__len__()//iters_per_epoch, 
    verbose   = True,
    callbacks = [cp_csv0, cp_callback]
)

Epoch 1/100
<class 'dict'> <class 'dict'>


/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Casting dense adjacency matrix to SparseTensor.This can be an expensive operation. 
  return py_builtins.overload_of(f)(*args)


train_step | pred: Tensor("Any:0", shape=(), dtype=bool) (None, 1)
train_step | loss: Tensor("Any_1:0", shape=(), dtype=bool) Tensor("Mean_1:0", shape=(), dtype=float32)
<class 'dict'> <class 'dict'>
train_step | pred: Tensor("Any:0", shape=(), dtype=bool) (None, 1)
train_step | loss: Tensor("Any_1:0", shape=(), dtype=bool) Tensor("Mean_1:0", shape=(), dtype=float32)


<class 'networkx.utils.decorators.argmap'> compilation 9:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/root/digital-twin/wireless_digital_twin/utils/datagen.py:238: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(self.graph_topology_undirected)
2022-12-23 05:57:25.684416: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x277e77f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-23 05:57:25.684476: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2022-12-23 05:57:25.684492: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2022-12-23 05:57:25.690810: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabli

241/241 [==============================] - ETA: 0s - loss: 175217.3663 - label/mean/delay: 431.5913 - prediction/mean/delay: 119.3360{'n_paths': <tf.Tensor 'IteratorGetNext:7' shape=() dtype=int32>, 'n_links': <tf.Tensor 'IteratorGetNext:5' shape=() dtype=int32>, 'n_nodes': <tf.Tensor 'IteratorGetNext:6' shape=() dtype=int32>, 'n_total': <tf.Tensor 'IteratorGetNext:8' shape=() dtype=int32>, 'paths_to_links': <tf.Tensor 'IteratorGetNext:13' shape=(None,) dtype=int32>, 'links_to_paths': <tf.Tensor 'IteratorGetNext:4' shape=(None,) dtype=int32>, 'sequences_paths_links': <tf.Tensor 'IteratorGetNext:17' shape=(None,) dtype=int32>, 'links_to_nodes': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int32>, 'nodes_to_links': <tf.Tensor 'IteratorGetNext:10' shape=(None,) dtype=int32>, 'sequences_links_nodes': <tf.Tensor 'IteratorGetNext:15' shape=(None,) dtype=int32>, 'paths_to_nodes': <tf.Tensor 'IteratorGetNext:14' shape=(None,) dtype=int32>, 'nodes_to_paths': <tf.Tensor 'IteratorGetNext:11

...

<class 'networkx.utils.decorators.argmap'> compilation 9:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/root/digital-twin/wireless_digital_twin/utils/datagen.py:238: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(self.graph_topology_undirected)



Epoch 2: saving model to ./training_1/cp.ckpt
241/241 [==============================] - 11s 48ms/step - loss: 32760.5025 - label/mean/delay: 429.3624 - prediction/mean/delay: 396.5616 - val_loss: 3802.7466 - val_label/mean/delay: 452.6000 - val_prediction/mean/delay: 417.8492
Epoch 3/100
241/241 [==============================] - ETA: 0s - loss: 31025.3004 - label/mean/delay: 437.6635 - prediction/mean/delay: 418.8022

<class 'networkx.utils.decorators.argmap'> compilation 9:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/root/digital-twin/wireless_digital_twin/utils/datagen.py:238: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(self.graph_topology_undirected)



Epoch 3: saving model to ./training_1/cp.ckpt
241/241 [==============================] - 12s 48ms/step - loss: 30969.3686 - label/mean/delay: 437.8628 - prediction/mean/delay: 418.7514 - val_loss: 3475.2207 - val_label/mean/delay: 452.6000 - val_prediction/mean/delay: 423.7196
Epoch 4/100
159/241 [==================>...........] - ETA: 3s - loss: 22704.3401 - label/mean/delay: 428.7044 - prediction/mean/delay: 412.9221

KeyboardInterrupt: 

In [ ]:
from tqdm.auto import tqdm
import time 

metrics_mae = []
delay_true = []
delay_pred = []
cnt = 0
for inp, lab in datagens['test']:
    t_start = time.time()
    
    delay_true.append(lab['drops'])
    delay_pred.append(model(inp).numpy().squeeze())
    
    t_end = time.time()
    cnt +=1
    print(cnt, 'used time {}s'.format(t_end - t_start), end = '\r')
    
delay_true = np.concatenate(delay_true)
delay_pred = np.concatenate(delay_pred)
metrics_mae = np.abs(delay_pred - delay_true)

In [ ]:
err = metrics_mae/delay_true
np.nanmean(err[~np.isinf(err)]), np.nanstd(err[~np.isinf(err)])
np.nanmean(metrics_mae), np.nanstd(metrics_mae)

In [ ]:
delay_true.mean()

In [ ]:
nlist = [3, 5, 2, 10,4,7]
ncumu = np.concatenate([[i]*n for i,n in enumerate(nlist)])
print(ncumu)

ii = 20
ncumu[ii], ii-sum(nlist[:ncumu[ii]])

In [ ]:
plt.boxplot([delay_true, delay_pred])
# plt.plot(delay_pred[:200])

In [ ]:
metrics_mae = np.abs(delay_pred - delay_true)

(metrics_mae/delay_true).mean(), (metrics_mae/delay_true).std()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1)
pd.read_csv('training3.log')[['loss', 'val_loss']].plot(ax=ax)

plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')

from matplotlib import ticker
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 
formatter.set_powerlimits((-1,1)) 
ax.yaxis.set_major_formatter(formatter) 

In [ ]:
# Test1
r_.evaluate(valid_data_generator)

In [ ]:
# Tetst2
r_.evaluate(test_data_generator)